In [ ]:
!git clone https://github.com/abewley/sort.git
!pip install scikit-image
!pip install filterpy
!pip install lap
!pip install ultralytics
!pip install easyocr
from ultralytics import YOLO
import cv2

import util
from sort.sort import *
from util import get_car, read_license_plate, write_csv

Cloning into 'sort'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 208 (delta 2), reused 2 (delta 1), pack-reused 202 (from 1)
Receiving objects: 100% (208/208), 1.20 MiB | 6.23 MiB/s, done.
Resolving deltas: 100% (74/74), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=25d53f8a3024bda46571456e2786285c427d2f8d1b0c436b61d2111546b54baf
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628943 sha256=e5bb90ea153633fea1bc64c166bd7f5511552d7df822d

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
results = {}

mot_tracker = Sort()

In [ ]:
coco_model = YOLO('yolov8n.pt')
license_plate_detector = YOLO('license_plate_detector.pt')

In [ ]:
cap = cv2.VideoCapture('sample.mp4')

vehicles = [2, 3, 5, 7]

In [ ]:
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}

# write results
write_csv(results, 'test.csv')

Streaming output truncated to the last 5000 lines.

0: 384x640 2 license_plates, 171.6ms
Speed: 4.1ms preprocess, 171.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 28 cars, 1 truck, 175.6ms
Speed: 8.7ms preprocess, 175.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 183.4ms
Speed: 4.2ms preprocess, 183.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 27 cars, 1 truck, 185.5ms
Speed: 5.8ms preprocess, 185.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 license_plates, 191.6ms
Speed: 4.1ms preprocess, 191.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 cars, 1 truck, 209.6ms
Speed: 4.2ms preprocess, 209.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 178.0ms
Speed: 7.4ms preprocess, 178.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640